In [3]:
#Function evaluate_poly evaluates a given derivative of a polynomial at a certain value
#Assumptions
# One dimensional polynomial
#Inputs
# coeffs - a vector of the polynomial coefficients that form the polynomial from c1 + c2*t + c3*t^2 + ...
# deriv - the number representing the ith derivative of the polynomial, a value of 0 means the 0th derivative
# t - the value(s) at which the derivative is evaluated at
#Outputs
# x - the value of the polynomial
function evaluate_poly(coeffs::Vector{Float64}, deriv::Int64, t)
    #see how many coefficients there are
    p = length(coeffs);
    #initialize x to zero
    x = 0;
    #loop as many times are needed to go through the polynomial after it has been derived
    for e = 0:(p-deriv-1)
        #the looping variable will start at zero since the polynomial's first power will start with at 0 no matter what
        #create the coefficient indexer by adding one to the looping variable and andding the deriv number
        ci = e + deriv +1;
        #calculate the derivative caused constants by using factorials
        d = factorial(e + deriv) / factorial(e);
        #combine the terms through multiplication and add to x
        x += d*coeffs[ci]*t.^e
        #repeat the loop
    end
    #return the end value
    return x;
end



#Function verifyActuateablePath checks if the smooth path is feasible given robots limits
#Assumptions
# Robot follows the quadrotor model set out by Minimum  Snap  Trajectory  Generation  and  Control  for  Quadrotors - Mellinger
# Euler angles of Z-X-Y
# Polynomials of the same degree
#Inputs
# solution - an object containing points, and times
# max_vel - the maximum velocity that the robot is limited to in ros
# max_accel - the maximum position acceleration that the robot is limited to 
# max_jerk - the maximum position jerk that the robot is limited to
# max_motor_rpm - the maximum rpm that a motor can get
#Outputs
# did_pass - a boolean that is true when the path passes
# timeProbv - a vector of times where motion is infeasible based on velocity
# timeProbm - a vector of times where motion is infeasible based on velocity
function verifyActuateablePath(solution::PolySol, max_vel::Float64, max_accel::Float64, max_jerk::Float64, max_motor_rpm::Float64)
    #Extract important information from the solution object
    degree = 2 + 2*(solution.params.cont_order-1) #create the degree of each polynomial assuming 2 pts for each
    num_poly = solution.num_segs;
    xcoeffs = solution.x_coeffs;
    ycoeffs = solution.y_coeffs;
    zcoeffs = solution.z_coeffs;
    pcoeffs = solution.p_coeffs;
    time_vec = solution.times;
    timeProbv = zeros(0,1);
    timeProbm = zeros(0,1);
    #Set some important constants for this function
    did_pass = true;
    time_res = 100 #The resolution segments 
    red_degree_by = 2; #The two is because the we are taking the derivative twice
    red_degree = degree - red_degree_by; #create a reduced degree to used for the dd and ddd calcs
    #Needed Constants for calculating the motor rpm to be on path
    z_w = [0,0,1]; #the up vector for the world frame
    Jxx = 0.0036404; #Values for the inertia matrix maybe can take from somewhere
    Jyy = 0.0053670;
    Jzz = 0.0030522;
    Jxy = 2.9264e-6;
    Jxz = 2.3411e-5;
    Jyz = 0.0001756;
    I = [Jxx Jxy Jxz
        Jxy Jyy Jyz
        Jxz Jyz Jzz];
    gravity = 9.8; 
    u2rpm_1 = 5096839.959225280;  #Values for an inverted allocation matrix to convert input into rpm
    u2rpm_2 = 51485858.53986801;  #Can probably get from code already coded
    u2rpm_3 = -51485858.53986803;
    u2rpm_4 = 330817430.2740964;
    u2rpms = [u2rpm_1 u2rpm_2 u2rpm_3 u2rpm_4 #inverted allocation matrix
        u2rpm_1 -u2rpm_2 -u2rpm_3 u2rpm_4
        u2rpm_1 u2rpm_2 -u2rpm_3 -u2rpm_4
        u2rpm_1 -u2rpm_2 u2rpm_3 -u2rpm_4];
    mass = 0.800; # in kilograms

    #####################
    #create variables to hold values initialize containers with zeros
    #Needed for z_B and u1
    xdd = zeros(num_poly*time_res);
    ydd = xdd;
    zdd = xdd;
    #Needed for speed check
    xd = xdd;
    yd = xdd;
    zd = xdd;
    #Needed for good plotting
    x = zeros(Float64,0,1);
    y = zeros(Float64,0,1);
    z = zeros(Float64,0,1);
    #Needed for a_dot
    xddd = xdd;
    yddd = xdd;
    zddd = xdd;
    #Needed for the inputs to the copter
    u1_vec = xdd;
    u2_vec = xdd;
    u3_vec = xdd;
    u4_vec = xdd;
    #Create a time vector for reporting purpose
    timeRep = zeros(Float64,0,1);
    #Needed initializations
    z_B = 0; #variable to hold the up body vector of copter
    #Needed for coefficients and angular acceleration
    yawdd = xdd;
    yaw = xdd;
    #create containers for problem points
    xprob = zeros(0,1);
    yprob = zeros(0,1);
    xddprob = zeros(0,1);
    yddprob = zeros(0,1);
    xdddprob = zeros(0,1);
    ydddprob = zeros(0,1);
    xddddprob = zeros(0,1);
    yddddprob = zeros(0,1);
    ######################Simple Method#####################################
    #Create a figure for debugging plots
    figure();
    #For each segment
    for seg = 1:num_poly
        #Create indexing range for the coefficients
        index_range = (1:degree)+degree*(seg-1);
        #Get the coefficients of the polynomial
        xcoeffs_s = xcoeffs[index_range];
        ycoeffs_s = ycoeffs[index_range];
        zcoeffs_s = zcoeffs[index_range];
        pcoeffs_s = pcoeffs[index_range];
        #Create the time vector for the polynomials, because of the way the problem is set up
        # every polynomial must be evaluated from 0 to the end of its range and then shifted
        t = collect(linspace(0,time_vec[seg+1]-time_vec[seg],time_res));
        #Generate a time vector for plotting and reporting
        timeRep = [timeRep; t+time_vec[seg]]
        #Calculate positions
        x = [x; evaluate_poly(xcoeffs_s,0,t)];
        y = [y; evaluate_poly(ycoeffs_s,0,t)];
        z = [z; evaluate_poly(zcoeffs_s,0,t)];
        #Calculate the velocities
        xd = evaluate_poly(xcoeffs_s,1,t);
        yd = evaluate_poly(ycoeffs_s,1,t);
        zd = evaluate_poly(zcoeffs_s,1,t);
        #Calculate the velocities that the robot will experience
        total_vel = sqrt(xd.^2 + yd.^2 + zd.^2);
        #Calculate the accelerations
        xdd = evaluate_poly(xcoeffs_s,2,t);
        ydd = evaluate_poly(ycoeffs_s,2,t);
        zdd = evaluate_poly(zcoeffs_s,2,t);
        yawdd = evaluate_poly(xcoeffs_s,2,t);
        #Calculate the total position acceleration
        total_accel = sqrt(xdd.^2 + ydd.^2 + zdd.^2);
        #Calculate the jerks
        xddd = evaluate_poly(xcoeffs_s,3,t);
        yddd = evaluate_poly(ycoeffs_s,3,t);
        zddd = evaluate_poly(zcoeffs_s,3,t);
        #Calculate the total position jerk
        total_jerk = sqrt(xddd.^2 + yddd.^2 + zddd.^2);
        #Calculate the snaps
        #Compare against the limits and store time and points
        #Velocity Limits
        timeProbv = [timeProbv; timeRep[find(total_vel .> max_vel)]];
        xprob = [xprob; evaluate_poly(xcoeffs_s,0,t[find(total_vel .> max_vel)])];
        yprob = [yprob; evaluate_poly(ycoeffs_s,0,t[find(total_vel .> max_vel)])];
        #Acceleration Limits
        timeProbv = [timeProbv; timeRep[find(total_accel .> max_accel)]];
        xddprob = [xddprob; evaluate_poly(xcoeffs_s,0,t[find(total_accel .> max_accel)])];
        yddprob = [yddprob; evaluate_poly(ycoeffs_s,0,t[find(total_accel .> max_accel)])];
        #Jerk Limits
        timeProbv = [timeProbv; timeRep[find(total_jerk .> max_jerk)]];
        xdddprob = [xdddprob; evaluate_poly(xcoeffs_s,0,t[find(total_jerk .> max_jerk)])];
        ydddprob = [ydddprob; evaluate_poly(ycoeffs_s,0,t[find(total_jerk .> max_jerk)])];
        #Note what time and position it occurs
        #Plot the graphs for debugging
        #figure();
        #Plot Problem points on position graph
        plot(t+time_vec[seg],zd);
    end
    #Print debugging information
    println(xprob)
    #Plot points where path is infeasible
    if(!isempty(timeProbv))
        figure(); 
        xlabel("X"); ylabel("Y"); title("Portion of the Path that has Feasibility Problems");
        plot(x,y,color=:gray,linestyle=":"); #Path
        scatter(xprob, yprob); # Velocity Prob
        scatter(xddprob, yddprob, color=:green);
        scatter(xdddprob, ydddprob, color=:red);
        legend(["Path","Velocity", "Acceleration", "Jerk"], loc=0)
    end
    #Return time locations where limits were exceeded.
    return timeProbv;
    ########################Complex method##################################
    #In the complex method the copter is only limited by the motor, arbitrary
    # limits can be applied should theoretically be unnecessary
    #For each segment
    #Calculate second derivatives of segments
    #Calculate z_B - body frame up vector
    #Calculate x_c
    #Calculate y_B
    #Calculate x_B
    #Calculate third derivatives
    #Calculate u1
    #Calculate h_w
    #Calculate yaws
    #Calculate w_bw
    #Calculate u2 u3 u4
    #Calculate the rpms
    #Compare to copter capabilities
    #Note what times and positions did the path become infeasible
end

LoadError: LoadError: UndefVarError: PolySol not defined
while loading In[3], in expression starting on line 45